In [3]:
#Importando paquetes
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
import urllib.request, urllib.parse, urllib.error
from requests.packages.urllib3.util.retry import Retry
import pandas as pd

# **1. WEB SCRAPING SENAHMI**

El SENAHMI (Servicio Nacional de Meteorología e Hidrología del Perú) es la entidad encargada de proporcionar información meteorológica y climatológica en el Perú. El web scraping, una técnica que implica la extracción automatizada de datos de páginas web, se utilizará para recopilar y organizar esta valiosa información de manera eficiente. El enfoque se centrará en respetar los términos de servicio del sitio, analizar la estructura HTML para identificar los elementos relevantes y utilizar herramientas de web scraping. Este proceso permitirá obtener datos actualizados de la SENAHMI, facilitando su posterior análisis y aplicación en diversas áreas.

### **Analizando HTML con BeautifulSoup**

In [4]:
# Ingresar URL
url = 'https://www.senamhi.gob.pe/?p=pronostico-meteorologico'

# Leer el contenido HTML de la URL
html = urllib.request.urlopen(url).read()

# Crear un objeto BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Imprimir la sección <head> del HTML
# print(soup.head)

### **Extracción de Datos Meteorológicos por Ciudad**

In [5]:
# Encuentra todas las divisiones que contienen la información de la ciudad
city_divs = soup.find_all('div', class_='col-lg-12 p-2 m-0', style='background-color:lightblue;')

# Listas para almacenar los datos antes de crear el DataFrame
cities = []
days = []
temperature_highs = []
temperature_lows = []
descriptions = []

**Captura Eficiente de Datos Meteorológicos por Ciudad**

En este paso, iteramos sobre las divisiones de la ciudad, extraemos la información necesaria y la almacenamos en listas.

In [6]:
for city_div in city_divs:
    # Extrae el nombre de la ciudad
    city_name_elem = city_div.find('span', class_='nameCity')
    if city_name_elem:
        city_name = city_name_elem.text.strip()

        # Encuentra el contenedor de información del clima para la ciudad actual
        td_element = city_div.find_next('td')

        try:
            # Extrae la información del clima para cada día
            for day_info in td_element.find_all('div', class_='row m-3', recursive=False):
                day = day_info.find('div', class_='col-sm-3').text.strip()
                temperature_high = day_info.find('div', class_='col-sm-1 text-danger').strong.text.strip()
                temperature_low = day_info.find('div', class_='col-sm-1 text-primary').strong.text.strip()
                description = day_info.find('div', class_='col-sm-6').text.strip()

                # Almacena la información en las listas
                cities.append(city_name)
                days.append(day)
                temperature_highs.append(temperature_high)
                temperature_lows.append(temperature_low)
                descriptions.append(description)

        except AttributeError as e:
            # Almacena información de error en listas
            cities.append(city_name)
            days.append("Error")
            temperature_highs.append("Error")
            temperature_lows.append("Error")
            descriptions.append("Error")

# Imprime las listas después de la iteración
#print("Cities:", cities)
#print("Days:", days)
#print("Temperature Highs:", temperature_highs)
#print("Temperature Lows:", temperature_lows)
#print("Descriptions:", descriptions)


**Estructuración de Datos Meteorológicos en un DataFrame**

In [7]:
# Crea un DataFrame de pandas con los datos recopilados
df = pd.DataFrame({
    'City': cities,
    'Day': days,
    'Temperature High': temperature_highs,
    'Temperature Low': temperature_lows,
    'Description': descriptions
})

# Muestra el DataFrame
df

,City,Day,Temperature High,Temperature Low,Description
0,BAGUA GRANDE - AMAZONAS,"sábado, 13 de enero",19ºC,11ºC,Cielo nublado parcial hacia la madrugada a cie...
1,BAGUA GRANDE - AMAZONAS,"domingo, 14 de enero",19ºC,11ºC,Cielo nublado parcial a cielo nublado durante ...
2,BAGUA GRANDE - AMAZONAS,"lunes, 15 de enero",19ºC,11ºC,Cielo nublado parcial por la mañana a cielo nu...
3,BAGUA GRANDE - AMAZONAS,"martes, 16 de enero",19ºC,11ºC,Cielo nublado con viento moderado por la mañan...
4,BAGUA GRANDE - AMAZONAS,"miércoles, 17 de enero",19ºC,11ºC,Cielo nublado parcial entre cielo nublado dura...
...,...,...,...,...,...
1301,PUERTO ESPERANZA - UCAYALI,"domingo, 14 de enero",31ºC,23ºC,Cielo nublado variando a cielo nublado parcial...
1302,PUERTO ESPERANZA - UCAYALI,"lunes, 15 de enero",30ºC,23ºC,Cielo cubierto a cielo nublado durante el día ...
1303,PUERTO ESPERANZA - UCAYALI,"martes, 16 de enero",30ºC,23ºC,Cielo cubierto a cielo nublado durante el día ...
1304,PUERTO ESPERANZA - UCAYALI,"miércoles, 17 de enero",32ºC,23ºC,Cielo nublado variando a cielo nublado parcial...


### **Mejora en la Estructura del DataFrame para Evitar Repetición de Ciudades por Días Meteorológicos**

En el DataFrame previo, las ciudades se repiten debido a múltiples registros de días meteorológicos por ciudad. Para solucionarlo, se propone reorganizar el DataFrame colocando cada día meteorológico como una columna individual. Esto se logra mediante una operación de pivote, evitando así la repetición de filas para la misma ciudad.

**Preparación para el Análisis Meteorológico Detallado**

In [8]:
# Continúa con el análisis y la extracción de datos si se obtiene el HTML correctamente
soup = BeautifulSoup(html, 'html.parser')

# Encuentra todas las divisiones que contienen la información de la ciudad
city_divs = soup.find_all('div', class_='col-lg-12 p-2 m-0', style='background-color:lightblue;')

# Lista para almacenar los datos antes de crear el DataFrame
combined_info = []

**Recopilación Integral de Datos Meteorológicos por Ciudad**

In [9]:
for city_div in city_divs:
    # Extrae el nombre de la ciudad
    city_name_elem = city_div.find('span', class_='nameCity')
    if city_name_elem:
        city_name = city_name_elem.text.strip()

        # Encuentra el contenedor de información del clima para la ciudad actual
        td_element = city_div.find_next('td')

        try:
            # Intenta extraer la información de cada día
            day_info_list = td_element.find_all('div', class_='row m-3', recursive=False)
        except AttributeError as e:
            # Maneja errores al extraer información de días
            print(f"Error extracting data for {city_name}: {e}")
            day_info_list = []

        # Verificación adicional antes de procesar la información del día
        if day_info_list:
            combined_info_dict = {'City': city_name}

            for i, day_info in enumerate(day_info_list):
                # Extrae información del día, como temperatura máxima, mínima y descripción
                day = day_info.find('div', class_='col-sm-3').text.strip()
                temperature_high = day_info.find('div', class_='col-sm-1 text-danger').strong.text.strip()
                temperature_low = day_info.find('div', class_='col-sm-1 text-primary').strong.text.strip()
                description = day_info.find('div', class_='col-sm-6').text.strip()

                # Almacena la información en el diccionario
                combined_info_dict[f'Day {i+1}'] = f"{day}: Máx {temperature_high}, Mín {temperature_low}, {description}"

            # Agrega el diccionario a la lista
            combined_info.append(combined_info_dict)

Error extracting data for SAN ALEJANDRO - UCAYALI: 'NoneType' object has no attribute 'find_all'


Este código extrae datos de manera cuidadosa para cada ciudad, identificando el nombre y localizando el contenedor climático. Intenta extraer la información diaria, manejando errores con elegancia y asegurando el éxito en la obtención de los datos del día.

**Representación Detallada de Pronósticos Diarios**

In [10]:
# Crea un DataFrame de pandas con los datos recopilados
df_por_dia = pd.DataFrame(combined_info)

# Muestra el DataFrame
df_por_dia

,City,Day 1,Day 2,Day 3,Day 4,Day 5
0,BAGUA GRANDE - AMAZONAS,"sábado, 13 de enero: Máx 19ºC, Mín 11ºC, Cielo...","domingo, 14 de enero: Máx 19ºC, Mín 11ºC, Ciel...","lunes, 15 de enero: Máx 19ºC, Mín 11ºC, Cielo ...","martes, 16 de enero: Máx 19ºC, Mín 11ºC, Cielo...","miércoles, 17 de enero: Máx 19ºC, Mín 11ºC, Ci..."
1,CHACHAPOYAS - AMAZONAS,"sábado, 13 de enero: Máx 32ºC, Mín 22ºC, Cielo...","domingo, 14 de enero: Máx 32ºC, Mín 22ºC, Ciel...","lunes, 15 de enero: Máx 32ºC, Mín 22ºC, Cielo ...","martes, 16 de enero: Máx 33ºC, Mín 22ºC, Cielo...","miércoles, 17 de enero: Máx 33ºC, Mín 22ºC, Ci..."
2,CHIRIACO - AMAZONAS,"sábado, 13 de enero: Máx 24ºC, Mín 17ºC, Cielo...","domingo, 14 de enero: Máx 24ºC, Mín 17ºC, Ciel...","lunes, 15 de enero: Máx 25ºC, Mín 16ºC, Cielo ...","martes, 16 de enero: Máx 26ºC, Mín 16ºC, Cielo...","miércoles, 17 de enero: Máx 26ºC, Mín 16ºC, Ci..."
3,PEDRO RUIZ - AMAZONAS,"sábado, 13 de enero: Máx 31ºC, Mín 22ºC, Cielo...","domingo, 14 de enero: Máx 31ºC, Mín 22ºC, Ciel...","lunes, 15 de enero: Máx 32ºC, Mín 22ºC, Cielo ...","martes, 16 de enero: Máx 32ºC, Mín 22ºC, Cielo...","miércoles, 17 de enero: Máx 32ºC, Mín 22ºC, Ci..."
4,SANTA MARIA DE NIEVA - AMAZONAS,"sábado, 13 de enero: Máx 13ºC, Mín 9ºC, Cielo ...","domingo, 14 de enero: Máx 14ºC, Mín 8ºC, Cielo...","lunes, 15 de enero: Máx 15ºC, Mín 7ºC, Cielo n...","martes, 16 de enero: Máx 16ºC, Mín 7ºC, Cielo ...","miércoles, 17 de enero: Máx 15ºC, Mín 8ºC, Cie..."
...,...,...,...,...,...,...
268,AGUAYTIA - UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 22ºC, Cielo...","domingo, 14 de enero: Máx 30ºC, Mín 22ºC, Ciel...","lunes, 15 de enero: Máx 32ºC, Mín 22ºC, Cielo ...","martes, 16 de enero: Máx 32ºC, Mín 22ºC, Cielo...","miércoles, 17 de enero: Máx 32ºC, Mín 22ºC, Ci..."
269,ATALAYA - UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 22ºC, Cielo...","domingo, 14 de enero: Máx 31ºC, Mín 22ºC, Ciel...","lunes, 15 de enero: Máx 30ºC, Mín 22ºC, Cielo ...","martes, 16 de enero: Máx 32ºC, Mín 22ºC, Cielo...","miércoles, 17 de enero: Máx 31ºC, Mín 22ºC, Ci..."
270,CURIMANÁ - UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 23ºC, Cielo...","domingo, 14 de enero: Máx 32ºC, Mín 23ºC, Ciel...","lunes, 15 de enero: Máx 30ºC, Mín 23ºC, Cielo ...","martes, 16 de enero: Máx 31ºC, Mín 23ºC, Cielo...","miércoles, 17 de enero: Máx 31ºC, Mín 23ºC, Ci..."
271,PUCALLPA - UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 21ºC, Cielo...","domingo, 14 de enero: Máx 32ºC, Mín 22ºC, Ciel...","lunes, 15 de enero: Máx 31ºC, Mín 22ºC, Cielo ...","martes, 16 de enero: Máx 31ºC, Mín 22ºC, Cielo...","miércoles, 17 de enero: Máx 31ºC, Mín 22ºC, Ci..."


### **Organización Detallada de Datos Meteorológicos por Día, Ciudad y Región**

Después de dividir la información por día, notamos que está concentrada en una celda, dificultando la visualización. Proponemos crear columnas separadas para cada observación, con fecha, temperaturas y descripción del día. Además, sugerimos añadir dos columnas para la ciudad y la región, simplificando así la organización y el análisis por ubicación.

**Preparación para el Análisis Meteorológico Detallado**

In [11]:
# Utiliza BeautifulSoup para analizar el HTML de la página web
soup = BeautifulSoup(html, 'html.parser')

# Encuentra todas las divisiones que contienen la información de la ciudad
city_divs = soup.find_all('div', class_='col-lg-12 p-2 m-0', style='background-color:lightblue;')

# Lista para almacenar los datos antes de crear el DataFrame
combined_info = []

**Iteración Cuidadosa para Detalles Meteorológicos por Ciudad**

In [12]:
# Itera sobre cada división que contiene información de la ciudad
for city_div in city_divs:
    # Extrae el nombre de la ciudad
    city_name_elem = city_div.find('span', class_='nameCity')
    if city_name_elem:
        # Limpia y almacena el nombre de la ciudad
        city_name = city_name_elem.text.strip()

        # Encuentra el contenedor de información del clima para la ciudad actual
        td_element = city_div.find_next('td')

        try:
            # Intenta extraer la información de cada día
            day_info_list = td_element.find_all('div', class_='row m-3', recursive=False)
        except AttributeError as e:
            # Maneja errores al extraer información de días
            print(f"Error extracting data for {city_name}: {e}")
            day_info_list = []

        # Verificación adicional antes de procesar la información del día
        if day_info_list:
            # Crea un diccionario para almacenar la información de la ciudad y sus días
            combined_info_dict = {'Ciudad': city_name}

            # Itera sobre la información de cada día
            for i, day_info in enumerate(day_info_list):
                # Extrae información específica del día
                day = day_info.find('div', class_='col-sm-3').text.strip()
                temperature_high = day_info.find('div', class_='col-sm-1 text-danger').strong.text.strip()
                temperature_low = day_info.find('div', class_='col-sm-1 text-primary').strong.text.strip()
                description = day_info.find('div', class_='col-sm-6').text.strip()

                # Almacena la información en el diccionario
                combined_info_dict[f'Day {i+1}'] = f"{day}: Máx {temperature_high}, Mín {temperature_low}"
                combined_info_dict[f'Description {i+1}'] = description

            # Agrega el diccionario a la lista de información combinada
            combined_info.append(combined_info_dict)

Error extracting data for SAN ALEJANDRO - UCAYALI: 'NoneType' object has no attribute 'find_all'


En esta sección, se analiza minuciosamente cada división que contiene datos de la ciudad. Se extrae y almacena el nombre de la ciudad, se localiza el contenedor de información climática y se intenta extraer la información diaria, gestionando con eficacia posibles errores.

**Estructuración y Organización Eficiente de Datos Meteorológicos**

In [13]:
# Crea un DataFrame de pandas con la información recopilada
df_senahmi = pd.DataFrame(combined_info)

# Divide la columna 'Ciudad' en 'Ciudad' y 'Región'
df_senahmi[['Ciudad', 'Región']] = df_senahmi['Ciudad'].str.split('-', n=1, expand=True)

# Reordena las columnas del DataFrame
df_senahmi = df_senahmi[['Ciudad', 'Región'] + [col for col in df_senahmi.columns if col not in ['Ciudad', 'Región']]]

# Guardar el DataFrame en un archivo CSV con la codificación UTF-8
df_senahmi.to_csv('pronostico_meteorologico.csv', index=False, encoding='utf-8')

# Muestra el DataFrame resultante
df_senahmi

,Ciudad,Región,Day 1,Description 1,Day 2,Description 2,Day 3,Description 3,Day 4,Description 4,Day 5,Description 5
0,BAGUA GRANDE,AMAZONAS,"sábado, 13 de enero: Máx 19ºC, Mín 11ºC",Cielo nublado parcial hacia la madrugada a cie...,"domingo, 14 de enero: Máx 19ºC, Mín 11ºC",Cielo nublado parcial a cielo nublado durante ...,"lunes, 15 de enero: Máx 19ºC, Mín 11ºC",Cielo nublado parcial por la mañana a cielo nu...,"martes, 16 de enero: Máx 19ºC, Mín 11ºC",Cielo nublado con viento moderado por la mañan...,"miércoles, 17 de enero: Máx 19ºC, Mín 11ºC",Cielo nublado parcial entre cielo nublado dura...
1,CHACHAPOYAS,AMAZONAS,"sábado, 13 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado a cielo nublado parcial durante ...,"domingo, 14 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado parcial entre cielo nublado dura...,"lunes, 15 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado entre cielo nublado parcial dura...,"martes, 16 de enero: Máx 33ºC, Mín 22ºC",Cielo nublado a cielo nublado parcial durante ...,"miércoles, 17 de enero: Máx 33ºC, Mín 22ºC",Cielo nublado a cielo nublado parcial durante ...
2,CHIRIACO,AMAZONAS,"sábado, 13 de enero: Máx 24ºC, Mín 17ºC",Cielo nublado parcial por la mañana a cielo nu...,"domingo, 14 de enero: Máx 24ºC, Mín 17ºC",Cielo nublado parcial a cielo nublado durante ...,"lunes, 15 de enero: Máx 25ºC, Mín 16ºC",Cielo nublado parcial con viento moderado por ...,"martes, 16 de enero: Máx 26ºC, Mín 16ºC",Cielo nublado parcial entre cielo nublado dura...,"miércoles, 17 de enero: Máx 26ºC, Mín 16ºC",Cielo nublado parcial entre cielo nublado dura...
3,PEDRO RUIZ,AMAZONAS,"sábado, 13 de enero: Máx 31ºC, Mín 22ºC",Cielo nublado entre cielo nublado parcial dura...,"domingo, 14 de enero: Máx 31ºC, Mín 22ºC",Cielo nublado entre cielo nublado parcial dura...,"lunes, 15 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado parcial a cielo nublado durante ...,"martes, 16 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"miércoles, 17 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado entre cielo nublado parcial dura...
4,SANTA MARIA DE NIEVA,AMAZONAS,"sábado, 13 de enero: Máx 13ºC, Mín 9ºC",Cielo nublado parcial entre cielo con nubes di...,"domingo, 14 de enero: Máx 14ºC, Mín 8ºC",Cielo nublado parcial entre cielo con nubes di...,"lunes, 15 de enero: Máx 15ºC, Mín 7ºC",Cielo nublado parcial entre cielo con nubes di...,"martes, 16 de enero: Máx 16ºC, Mín 7ºC",Cielo con nubes dispersas por la mañana varian...,"miércoles, 17 de enero: Máx 15ºC, Mín 8ºC",Cielo con nubes dispersas variando a cielo nub...
...,...,...,...,...,...,...,...,...,...,...,...,...
268,AGUAYTIA,UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"domingo, 14 de enero: Máx 30ºC, Mín 22ºC",Cielo nublado parcial variando a cielo nublado...,"lunes, 15 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"martes, 16 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"miércoles, 17 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...
269,ATALAYA,UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 22ºC",Cielo nublado a cielo cubierto durante el día ...,"domingo, 14 de enero: Máx 31ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"lunes, 15 de enero: Máx 30ºC, Mín 22ºC",Cielo nublado a cielo cubierto durante el día ...,"martes, 16 de enero: Máx 32ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...,"miércoles, 17 de enero: Máx 31ºC, Mín 22ºC",Cielo nublado variando a cielo nublado parcial...
270,CURIMANÁ,UCAYALI,"sábado, 13 de enero: Máx 30ºC, Mín 23ºC",Cielo nublado variando a cielo nublado parcial...,"domingo, 14 de enero: Máx 32ºC, Mín 23ºC",Cielo con nubes dispersas variando a cielo nub...,"lunes, 15 de enero: Máx 30ºC, Mín 23ºC",Cielo nublado variando a cielo nublado parcial...,"martes, 16 de enero: Máx 31ºC, Mín 23ºC",Cielo cubierto variando a cielo nublado parcia...,"miércoles, 17 de enero: Máx 31ºC, Mín 23ºC",C